In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
from tqdm import tqdm

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class FrameInterpolationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(6, 64, 3, 1, 1),
                nn.ReLU(inplace=True),
                nn.Conv2d(64, 128, 3, 1, 1),
                nn.ReLU(inplace=True)
            ) for _ in range(3)
        ])
        self.resize = nn.Upsample(size=(90, 160), mode='bilinear', align_corners=False)
        self.fusion_conv = nn.Conv2d(384, 128, 3, 1, 1)
        self.upsample_conv = nn.ConvTranspose2d(128, 3, 3, 2, 1, 1, 1)

    def forward(self, x):
        feature_maps = [extractor(x) for extractor in self.feature_extractor]
        feature_maps_resized = [self.resize(fm) for fm in feature_maps]
        x = torch.cat(feature_maps_resized, 1)
        x = F.relu(self.fusion_conv(x))
        x = self.upsample_conv(x)
        return x

class FrameInterpolationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.num_frames_per_video = 190
        self.start_frame_number = 100000  

    def __len__(self):
        return self.num_frames_per_video * 25  

    def __getitem__(self, idx):
        frame_idx = idx % self.num_frames_per_video
        video_idx = idx // self.num_frames_per_video
        frame_number = self.start_frame_number + frame_idx

        frame_folder = self.root_dir
        frame1_path = os.path.join(frame_folder, f"frame_{frame_number :06d}.jpg")
        frame2_path = os.path.join(frame_folder, f"frame_{frame_number + 1:06d}.jpg")
        target_path = os.path.join(frame_folder, f"frame_{frame_number + 2:06d}.jpg")

        frame1, frame2, target = map(Image.open, (frame1_path, frame2_path, target_path))

        if self.transform:
            frame1, frame2, target = map(self.transform, (frame1, frame2, target))

        return frame1, frame2, target

model = FrameInterpolationModel().to(device)  # Move model to GPU
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Loading dataset...")
batch_size = 16 # Adjust as necessary
dataset = FrameInterpolationDataset(root_dir='C:\\Users\\adhik\\Frame Interpolation\\weather data', transform=transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print("Dataset loaded.")

print("Training model...")
num_epochs = 3  # Set the number of epochs here
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    epoch_loss = 0
    
    for batch_idx, (frame1, frame2, target) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}")):
        # Move data to GPU
        frame1, frame2, target = frame1.to(device), frame2.to(device), target.to(device)
        
        inputs = torch.cat((frame1, frame2), 1)
        optimizer.zero_grad()
        outputs = model(inputs)
        
        target_resized = F.interpolate(target, size=outputs.shape[2:], mode='bilinear', align_corners=False)
        loss = criterion(outputs, target_resized)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if batch_idx % 10 == 0:
            print(f'Batch {batch_idx}/{len(dataloader)}, Loss: {loss.item()}')

    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f'\nEpoch {epoch+1}/{num_epochs}, Average Loss: {avg_epoch_loss}', flush=True)


Loading dataset...
Dataset loaded.
Training model...

Epoch 1/3


Epoch 1:   0%|▏                                                                      | 1/297 [00:54<4:28:21, 54.40s/it]

Batch 0/297, Loss: 0.05597037822008133


Epoch 1:   4%|██▌                                                                   | 11/297 [08:01<3:09:52, 39.83s/it]

Batch 10/297, Loss: 0.019104108214378357


...

Batch 280/297, Loss: 0.00040703380363993347


Epoch 3:  98%|███████████████████████████████████████████████████████████████████▌ | 291/297 [6:31:13<07:52, 78.75s/it]

Batch 290/297, Loss: 0.0001468476402806118


Epoch 3: 100%|█████████████████████████████████████████████████████████████████████| 297/297 [6:38:58<00:00, 80.60s/it]


Epoch 3/3, Average Loss: 0.0007314673078721483


In [4]:
torch.save(model.state_dict(), 'frame_interpolation_model2.pth')

In [86]:
model_path = 'frame_interpolation_model2.pth'
model = FrameInterpolationModel().to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

FrameInterpolationModel(
  (feature_extractor): ModuleList(
    (0-2): 3 x Sequential(
      (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
  )
  (resize): Upsample(size=(90, 160), mode='bilinear')
  (fusion_conv): Conv2d(384, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (upsample_conv): ConvTranspose2d(128, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
)

In [87]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os

In [88]:
def interpolate_frame(frame1, frame2, alpha=0.5, scale_factor=2, output_path='interpolated_frame.png'):
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    
    # Load mod
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = FrameInterpolationModel().to(device)
    model_path = 'frame_interpolation_model2.pth'
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    frame1 = transform(frame1).unsqueeze(0).to(device)
    frame2 = transform(frame2).unsqueeze(0).to(device)

    # Interpolate frames
    inputs = torch.cat((frame1, frame2), 1)  # Concatenate along the channel dimension
    
    with torch.no_grad():
        output = model(inputs)
    
    # Upsample the output
    height, width = output.shape[2], output.shape[3]
    new_size = (width * scale_factor, height * scale_factor)
    output = F.interpolate(output, size=new_size, mode='bilinear', align_corners=False)
    
    # Convert to PIL Image
    output_image = transforms.ToPILImage()(output.squeeze(0).cpu())
    
    # Save the image
    output_image.save(output_path)
    print(f'Saved: {output_path}')
    
    return output_image



In [89]:
frame1_path = 'horse1.jpg'
frame2_path = 'horse2.jpg'
output_dir = 'interpolated_frames'
interpolated_frame = interpolate_frame(frame1, frame2, alpha=0.5, scale_factor=2, output_path='horseend.png')


Saved: horseend.png


In [71]:
def generate_interpolated_frames(frame1_path, frame2_path, output_dir, num_frames=900, scale_factor=2):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load input frames
    frame1 = Image.open(frame1_path)
    frame2 = Image.open(frame2_path)

    for i in range(num_frames):
        alpha = i / (num_frames - 1)  # Interpolation coefficient from 0.0 to 1.0
        
        # Define output path for each frame
        output_path = os.path.join(output_dir, f'frame_{i:04d}.png')
        
        # Generate interpolated frame and save it
        interpolate_frame(frame1, frame2, alpha, scale_factor, output_path)
    
    print(f'All {num_frames} frames have been saved in the directory: {output_dir}')


In [72]:
# frame1_path = 'frame_100000.jpg'
# frame2_path = 'frame_120676.jpg'
# output_dir = 'interpolated_frames'

In [73]:
generate_interpolated_frames(frame1_path, frame2_path, output_dir, num_frames=900, scale_factor=2)

Saved: interpolated_frames\frame_0000.png
Saved: interpolated_frames\frame_0001.png
Saved: interpolated_frames\frame_0002.png
Saved: interpolated_frames\frame_0003.png
Saved: interpolated_frames\frame_0004.png
Saved: interpolated_frames\frame_0005.png
Saved: interpolated_frames\frame_0006.png
Saved: interpolated_frames\frame_0007.png
Saved: interpolated_frames\frame_0008.png
Saved: interpolated_frames\frame_0009.png
Saved: interpolated_frames\frame_0010.png
Saved: interpolated_frames\frame_0011.png
Saved: interpolated_frames\frame_0012.png
Saved: interpolated_frames\frame_0013.png
Saved: interpolated_frames\frame_0014.png
Saved: interpolated_frames\frame_0015.png
Saved: interpolated_frames\frame_0016.png
Saved: interpolated_frames\frame_0017.png
Saved: interpolated_frames\frame_0018.png
Saved: interpolated_frames\frame_0019.png
Saved: interpolated_frames\frame_0020.png
Saved: interpolated_frames\frame_0021.png
Saved: interpolated_frames\frame_0022.png
Saved: interpolated_frames\frame_0